In [ ]:
!pip install roboflow ultralytics

In [ ]:
import torch

# Vérification du GPU
print(f"GPU disponible: {torch.cuda.is_available()}")
print(f"Nombre de GPUs: {torch.cuda.device_count()}")

In [ ]:
from roboflow import Roboflow

In [ ]:
rf = Roboflow(api_key="9T2oi9LTqqHLoG9rwQnx", model_format="yolov8")  # Remplacez par votre clé API (Roboflow > Workspace Settings)
dataset = rf.workspace().project("my-first-project-f1bpw-kbpj9").version(1).download(location="/content/my-datasets")

In [ ]:
dataset_path = "/kaggle/working/My-First-Project-1"
print("Contenu du dataset :")
!ls {dataset_path}

In [ ]:
# Vérification du fichier data.yaml
!cat {dataset_path}/data.yaml

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Modèle nano (peut être changé en yolov8s/m/l/x)

# Paramètres d'entraînement
config = {
    "data": f"{dataset_path}/data.yaml",
    "epochs": 100,
    "imgsz": 640,
    "batch": 16,
    "device": 0 if torch.cuda.is_available() else "cpu",
    "name": "fruits_vegetables_yolov8",
    "optimizer": "Adam",  # Peut être changé en 'SGD' si nécessaire
    "seed": 42
}

In [ ]:
# Lancement de l'entraînement
results = model.train(**config)

In [ ]:
# Evaluation du modèle sur l'ensemble de validation
print("\n⚡ Évaluation du modèle :")
metrics = model.val(
    data=f"{dataset_path}/data.yaml",
    split='valid',  # ou 'test' si disponible
    conf=0.5,     # seuil de confiance
    iou=0.50,      # seuil IoU
    device=0       #GPU
)

print(f"""
Résultats :
- mAP50: {metrics.box.map50:.3f}
- mAP50-95: {metrics.box.map:.3f}
- Précision: {metrics.box.precision.mean():.3f}
- Rappel: {metrics.box.recall.mean():.3f}
""")

In [ ]:
# Exemple sur une image de test
test_images_dir = "/kaggle/working/My-First-Project-1/test/images"
test_images = [f for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.png'))][:3]

print("🔍 Prédictions sur 3 images tests :")
for img_name in test_images:
    img_path = os.path.join(test_images_dir, img_name)
    results = model.predict(
        source=img_path,
        conf=0.5,
        save=True,
        save_txt=True,
        line_width=2
    )
    
    # Afficher l'image avec les prédictions
    result_img_path = f"/kaggle/working/runs/detect/predict/{img_name}"
    display(Image(filename=result_img_path, width=600))
    print(f"Image: {img_name} | Objets détectés: {len(results[0].boxes)}")

In [ ]:
!zip -r /kaggle/working/test_results.zip /kaggle/working/runs/detect/predict
print("✅ Résultats sauvegardés dans test_results.zip")

In [ ]:
from ultralytics import YOLO
model = YOLO('runs/detect/train/weights/best.pt')
model.export(format='tflite', imgsz=640, optimize=False, int8=False, dynamic=False, half=False)

### FPS

In [ ]:
from ultralytics import YOLO
import cv2
import time

# Charger le modèle YOLOv8 (votre fichier .pt)
model = YOLO("yolov8n.pt")  # Remplacez par votre modèle personnalisé

# Capturer la vidéo (webcam ou fichier)
cap = cv2.VideoCapture(0)  # Webcam (0) ou chemin vers une vidéo

# Variables pour le calcul du FPS
prev_time = 0
fps = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Mesurer le temps avant l'inférence
    start_time = time.time()

    # Faire la prédiction (inférence)
    results = model(frame, verbose=False)  # `verbose=False` désactive les logs

    # Calculer le FPS
    current_time = time.time()
    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    # Afficher le FPS sur l'image
    cv2.putText(
        frame, 
        f"FPS: {fps:.2f}", 
        (10, 30), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1, 
        (0, 255, 0), 
        2
    )

    # Afficher les résultats (optionnel)
    annotated_frame = results[0].plot()
    cv2.imshow("YOLOv8 FPS Test", annotated_frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()